In [16]:
import time
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.neural_network import MLPRegressor


class MaxStdScaler(BaseEstimator, TransformerMixin):
    def __init__(self, copy = True, factor = 1.0):
        self.factor = float(factor)
        self.copy = copy

    def fit(self, X):
        X = check_array(X, copy = self.copy, estimator = self, dtype = np.float64)
        self.scaler = np.max(np.std(X, axis = 0)) / self.factor
        return self

    def transform(self, X):
        X = check_array(X, copy = self.copy, estimator = self, dtype = np.float64)
        X /= self.scaler

        return X

    def inverse_transform(self, X, copy = None):
        if copy is None:
            copy = self.copy

        X = check_array(X, copy = copy, estimator = self, dtype = np.float64)
        X *= self.scaler

        return X

def net_train_and_predict(X_train, y_train, X_pred, alpha, random_state, verbose = True):
    start_time = time.time()

    scaler_x = MaxStdScaler()
    X_train = scaler_x.fit_transform(X_train)
    scaler_y = MaxStdScaler(factor = 15.0)
    y_train = scaler_y.fit_transform(y_train)

    regressor = MLPRegressor(hidden_layer_sizes = (100, 75, 50, 25), activation = 'relu', solver = 'sgd', learning_rate = 'adaptive', alpha = alpha, random_state = random_state)
    regressor.fit(X_train, y_train)
    #print("Loss: ", regressor.loss_)

    y_pred = regressor.predict(scaler_x.transform(X_pred))

    end_time = time.time()

    if verbose:
        print("Deep regressor traning and predicting finished. Time spent = {:.2f}s.".format(end_time - start_time))

    return y_pred 

In [1]:
import numpy as np
import h5py
import os
import random
from keras import optimizers

import tensorflow as tf
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint

from keras.initializers import glorot_uniform
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle

SEED_VAL = 24
os.environ['SEED_NN'] = str(SEED_VAL)
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
tf.set_random_seed(SEED_VAL)

Using TensorFlow backend.


In [2]:
N_SAMPLE = 25
TEST_SAMPLE_ID = 0

In [3]:
#Time-samples
#for all hdf5 files
all_hdf5_data = np.zeros((N_SAMPLE,12,650,650),dtype=np.uint16)


for i in range(N_SAMPLE):
    data_file = h5py.File(str(i)+'_level2a.h5', 'r')
    arr = data_file.get('bands')
   
    np_arr =  np.array(arr, dtype=np.uint16)
    
    all_hdf5_data[i] = np_arr
   




In [4]:
# labels 
# actually cholorophil-a values for regression
import pandas
import sys 

labels_index = np.zeros((N_SAMPLE * 10), dtype=np.float)

labels_name = np.zeros((N_SAMPLE * 10), dtype=np.float)

df = pandas.read_csv('SDD_Value.csv')
    #print(df['Label'])
labels_name = df['SDD_Value']
#print(type(labels_name))

for index in range(len(labels_name)):
    #print(labels_name[index])
    labels_index[index] = labels_name[index]; 
    #print(index, labels_index[index])


print(labels_index)

[0.33 0.28 0.19 0.24 0.17 0.24 0.3  0.68 0.59 0.35 0.29 0.3  0.34 0.4
 0.4  0.41 0.38 0.37 0.49 0.42 0.6  0.7  0.56 0.54 0.72 0.38 0.3  0.23
 0.26 0.18 0.28 0.31 0.72 0.6  0.38 0.3  0.31 0.31 0.38 0.42 0.36 0.38
 0.38 0.48 0.41 0.58 0.72 0.56 0.54 0.72 0.38 0.31 0.27 0.27 0.19 0.31
 0.3  0.74 0.59 0.38 0.3  0.31 0.32 0.38 0.41 0.37 0.39 0.35 0.48 0.42
 0.57 0.7  0.56 0.52 0.72 0.32 0.36 0.2  0.3  0.2  0.3  0.32 0.8  0.63
 0.33 0.31 0.29 0.31 0.39 0.39 0.36 0.38 0.36 0.5  0.43 0.6  0.7  0.58
 0.54 0.74 0.3  0.39 0.27 0.29 0.2  0.3  0.32 0.83 0.62 0.3  0.31 0.29
 0.32 0.4  0.41 0.34 0.36 0.37 0.47 0.45 0.59 0.68 0.56 0.54 0.72 0.45
 0.37 0.22 0.3  0.27 0.3  0.35 0.47 0.55 0.39 0.31 0.31 0.33 0.38 0.41
 0.34 0.39 0.38 0.46 0.43 0.59 0.72 0.56 0.54 0.72 0.38 0.33 0.28 0.25
 0.29 0.3  0.24 0.45 0.52 0.38 0.32 0.31 0.3  0.39 0.47 0.36 0.38 0.38
 0.48 0.45 0.6  0.74 0.57 0.56 0.72 0.37 0.39 0.3  0.29 0.29 0.32 0.33
 0.49 0.57 0.37 0.3  0.29 0.32 0.4  0.45 0.39 0.39 0.37 0.48 0.46 0.56
 0.72 0

In [5]:
#coordinate
s = pandas.read_csv('sensor_pos.csv')
#print(df['Label'])
S_POS_X = s['Y']
S_POS_Y = s['X']

print(S_POS_X)

0    537
1    427
2    340
3    263
4    165
5    107
6    172
7    249
8    337
9    447
Name: Y, dtype: int64


In [6]:
#patches from hdf5 file
#convert to numpy array
#n1 = np.array(n1, dtype=np.uint16)
n1 = np.zeros((12, 1), dtype= np.uint16)
#print (n1.shape)

patches_from_sensors = np.zeros((10*N_SAMPLE, 12 , 1), dtype = np.uint16)
#print(patches_from_sensors.shape, patches_from_sensors.dtype)
#print(patches_from_sensors[0])
new_feature_time = np.zeros((1), dtype=np.uint16)
# Sensor position (X, Y)
# 3X3 Patches 
# extract patch from data, starts (X-1,Y-1) ends ( X+1, Y+1)
# DO NOT FORGET!!!! Meaning of [x:y] is that the new array includes values at the X pos., but not at the Y pos.

# Sensor positions
#X = 10
#Y = 11
STEPS = 3
TOTAL_PX = 49
#patch = n1[:, X - STEPS: X + STEPS + 1, Y - STEPS: Y + STEPS + 1]
#print(patch.shape, patch)

#patches_from_sensors[0] = patch
#print(patches_from_sensors[0])
total = np.zeros((1), dtype=np.uint16)


    #n1 = pca_hdf5_data[i_sample] # After PCA data
    #n1 = ch0_hdf5_data[i_sample] # 1 channel data
    #print(i_sample,"Sample : ", n1.shape) 
    #extract 10 (# stations) patches for one sample(for one hdf5 file)
for i_s in range(len(S_POS_X)):
        X = S_POS_X[i_s]
        Y = S_POS_Y[i_s]
        for i_sample in range(N_SAMPLE):
            n1 = all_hdf5_data[i_sample] #original data
        # print("X - Y:" , X , Y )
        # Patch Shapes : [C,W,H]
            patch = n1[:, X - STEPS: X + STEPS + 1, Y - STEPS: Y + STEPS + 1] #for patch
        #print(i_s,"s: ", patch.shape)
        #for a pixel
        # If you would like to use more than one channel
        # You should change like patch = n1[ :, X: X+ 1, Y: Y + 1] 
       
        
            #print(patch.shape)
                
            patch = np.reshape(patch,(patch.shape[0],TOTAL_PX))
        #patch = np.append(patch, new_feature_time) #for time
            #print("Patch "  ,patch.shape)
        
            for index in range(TOTAL_PX):
                total = total + patch[:,index]
                #print(index , "Patch Part: ",patch[:,index:index + 1])
            #print("before:" , total)    
            total = total/TOTAL_PX
            print(total.shape)
            #patch[12] = i_s
            patch = np.reshape(total,(total.shape[0],1))
           
            total = np.zeros((1), dtype=np.uint16)
    
           # print("New Patch "  ,patch.shape)
            patches_from_sensors[i_sample + (25 * i_s)] = patch
        
    
print("Number of all samples : ",len(patches_from_sensors))
print(" shape: ",patches_from_sensors.shape)

(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,)
(12,

In [7]:
#patches from hdf5 file
#convert to numpy array
#n1 = np.array(n1, dtype=np.uint16)
n1 = np.zeros((12, 1), dtype= np.uint16)
#print (n1.shape)

patches_from_sensors = np.zeros((10*N_SAMPLE, 12 , 1), dtype = np.uint16)
#print(patches_from_sensors.shape, patches_from_sensors.dtype)
#print(patches_from_sensors[0])
new_feature_time = np.zeros((1), dtype=np.uint16)
# Sensor position (X, Y)
# 3X3 Patches 
# extract patch from data, starts (X-1,Y-1) ends ( X+1, Y+1)
# DO NOT FORGET!!!! Meaning of [x:y] is that the new array includes values at the X pos., but not at the Y pos.

# Sensor positions
#X = 10
#Y = 11

#patch = n1[:, X - STEPS: X + STEPS + 1, Y - STEPS: Y + STEPS + 1]
#print(patch.shape, patch)

#patches_from_sensors[0] = patch
#print(patches_from_sensors[0])
total = np.zeros((1), dtype=np.uint16)


    #n1 = pca_hdf5_data[i_sample] # After PCA data
    #n1 = ch0_hdf5_data[i_sample] # 1 channel data
    #print(i_sample,"Sample : ", n1.shape) 
    #extract 10 (# stations) patches for one sample(for one hdf5 file)
for i_s in range(len(S_POS_X)):
        X = S_POS_X[i_s]
        Y = S_POS_Y[i_s]
        print("X: , Y: ", X, Y)
        for i_sample in range(N_SAMPLE):
            n1 = all_hdf5_data[i_sample] #original data
        # print("X - Y:" , X , Y )
        # Patch Shapes : [C,W,H]
           #patch = n1[:, X - STEPS: X + STEPS + 1, Y - STEPS: Y + STEPS + 1] #for patch
        #print(i_s,"s: ", patch.shape)
        #for a pixel
        # If you would like to use more than one channel
        # You should change like patch = n1[ :, X: X+ 1, Y: Y + 1] 
       
        
            #print(patch.shape)
                
        #patch = np.append(patch, new_feature_time) #for time
            #print("Patch "  ,patch.shape)
        
          
                #print(index , "Patch Part: ",patch[:,index:index + 1])
            #print("before:" , total)    
          
            #patch[12] = i_s
            patch = n1[:, X: X+ 1, Y: Y + 1] 
        
            print(patch)
                
        #patch = np.reshape(patch,(patch.shape[0],1))
        #patch = np.append(patch, new_feature_time) #for time
        #print("Patch "  ,patch.shape)
        #patch[12] = i_s
            patch = np.reshape(patch,(patch.shape[0],1))
        #patnp.append([[1, 2, 3], [4, 5, 6]], [[7, 8, 9]], axis=0)
            patches_from_sensors[i_sample + (25 * i_s)] = patch
        
    
print("Number of all samples : ",len(patches_from_sensors))
print(" shape: ",patches_from_sensors.shape)

('X: , Y: ', 537, 235)
[[[ 630]]

 [[ 783]]

 [[1045]]

 [[ 899]]

 [[1023]]

 [[ 674]]

 [[ 692]]

 [[ 605]]

 [[ 575]]

 [[ 189]]

 [[ 427]]

 [[ 354]]]
[[[1014]]

 [[1012]]

 [[1461]]

 [[1172]]

 [[1689]]

 [[1124]]

 [[1098]]

 [[ 903]]

 [[ 849]]

 [[ 640]]

 [[ 659]]

 [[ 629]]]
[[[ 825]]

 [[ 842]]

 [[1170]]

 [[ 836]]

 [[1367]]

 [[ 955]]

 [[ 954]]

 [[ 719]]

 [[ 717]]

 [[ 389]]

 [[ 422]]

 [[ 391]]]
[[[ 884]]

 [[ 972]]

 [[1381]]

 [[1081]]

 [[1524]]

 [[1044]]

 [[1071]]

 [[ 878]]

 [[ 797]]

 [[ 201]]

 [[ 545]]

 [[ 467]]]
[[[466]]

 [[550]]

 [[867]]

 [[673]]

 [[958]]

 [[459]]

 [[439]]

 [[366]]

 [[268]]

 [[ 72]]

 [[132]]

 [[124]]]
[[[606]]

 [[612]]

 [[806]]

 [[630]]

 [[762]]

 [[455]]

 [[456]]

 [[400]]

 [[320]]

 [[231]]

 [[ 72]]

 [[ 45]]]
[[[126]]

 [[425]]

 [[752]]

 [[589]]

 [[668]]

 [[234]]

 [[220]]

 [[155]]

 [[ 81]]

 [[  1]]

 [[  1]]

 [[  2]]]
[[[232]]

 [[434]]

 [[649]]

 [[449]]

 [[444]]

 [[164]]

 [[148]]

 [[ 99]]

 [[ 60]]


In [8]:
import os
def write_txt(data,label, file_n):
    with open(file_n, 'a') as f1:
        for index in range(data.shape[0]):
            for index_i in range(12):
               #f1.write(data[index][index_i])
                f1.write('{: }'.format(data[index][index_i]))
                f1.write(',')
                
            f1.write('{: }'.format(label[index]))
           
            f1.write('\n')

    f1.close()

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

total_coef = 0.0
total_MSE = 0.0
total_MAE = 0.0

total_coef_RF = 0.0
total_MSE_RF = 0.0
total_MAE_RF = 0.0

for i in range(10):
    X_train = np.vstack((patches_from_sensors[0*N_SAMPLE:i*N_SAMPLE], patches_from_sensors[(i+1)*N_SAMPLE:10*N_SAMPLE,:,:]) )

    y_train = np.append(labels_index[0*N_SAMPLE:i*N_SAMPLE], labels_index[(i+1)*N_SAMPLE:10*N_SAMPLE] )

    X_train = np.reshape(X_train,  (X_train.shape[0], 12))
    #y_train = np.reshape(y_train, (y_train.shape[0], 1))


    #print("X_train", X_train.shape)
    #print("y_train", y_train.shape)

    X_test = patches_from_sensors[i*N_SAMPLE:(i+1)*N_SAMPLE,:,:]
    #X_test = np.append(X_test, patches_from_sensors[9:10,:,:])
    y_test =labels_index[i*N_SAMPLE:(i+1)*N_SAMPLE]

    X_test = np.reshape(X_test, (X_test.shape[0], 12))
    #y_test = np.reshape(y_test, (y_test.shape[0], 1))

    #print("X_test", X_test.shape)
    #print("y_test", y_test)
    
    """
   
    regr = RandomForestRegressor(n_estimators = 100, random_state=42).fit(X_train, y_train)
    print("---------------RF REG---------------------")
    print("---------------TEST - "  + str(i) +  "--------------------")
    # Make predictions using the testing set
    y_pred = regr.predict(X_test) #for SVR

    #print(y_pred)
    #print("Real Values : ",y_test)
    #print("Predictions : ", y_pred) 
    r = np.corrcoef(y_test, y_pred)

    # Correlation Coefficients
    print('Correlation Coef.: ',r[0, 1] )

    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_test, y_pred))
 
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_test, y_pred))
    print('Mean absolute error: %.2f'
          % mean_absolute_error(y_test, y_pred))
    
    total_coef_RF = total_coef_RF + r[0, 1]
    total_MSE_RF = total_MSE_RF + mean_squared_error(y_test, y_pred)
    total_MAE_RF = total_MAE_RF + mean_absolute_error(y_test, y_pred)
    
     """
    y_train = np.reshape(y_train,  (y_train.shape[0], 1))
   
    
    y_pred = net_train_and_predict(X_train, y_train, X_test, 5, SEED_VAL)
   


    print("\n---------MLP REG. RESULTS-----------\n")
    #print(y_pred)
    print("Real Values : ",(y_test))
    print("Predictions : ", (y_pred/100)) 
    r = np.corrcoef(y_test, y_pred)

    # Correlation Coefficients
    print('Correlation Coef.: ',r[0, 1] )
    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_test, y_pred/100))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_test, y_pred/100))
    print('Mean absolute error: %.2f'
          % mean_absolute_error(y_test, y_pred/100))

    total_coef = total_coef + r[0, 1]
    total_MSE = total_MSE + mean_squared_error(y_test, y_pred)
    total_MAE = total_MAE + mean_absolute_error(y_test, y_pred)
    
    
    
#write_txt(X_train, y_train, 'train_' + str(i) + '.arff')

#write_txt(X_test, y_test , 'test_' + str(i) + '.arff')
    
print(" avg_coef : ",total_coef_RF/10)
print(" avg_MSE : ",total_MSE_RF/10)
print(" avg_MAE : ",total_MAE_RF/10 )  


print("MLP avg_coef : ",total_coef/10)
print("MLP avg_MSE : ",total_MSE/10)
print("MLP avg_MAE : ",total_MAE/10 )  
    


Deep regressor traning and predicting finished. Time spent = 0.63s.

---------MLP REG. RESULTS-----------

('Real Values : ', array([0.33, 0.28, 0.19, 0.24, 0.17, 0.24, 0.3 , 0.68, 0.59, 0.35, 0.29,
       0.3 , 0.34, 0.4 , 0.4 , 0.41, 0.38, 0.37, 0.49, 0.42, 0.6 , 0.7 ,
       0.56, 0.54, 0.72]))
('Predictions : ', array([0.30981549, 0.3013948 , 0.30385165, 0.2915488 , 0.30830392,
       0.32954424, 0.36334081, 0.45173909, 0.40367853, 0.34832707,
       0.30479898, 0.31021605, 0.3130583 , 0.40615719, 0.47494984,
       0.4459705 , 0.34167352, 0.51026744, 0.54229309, 0.47843015,
       0.52009798, 0.64377774, 0.57041348, 0.66681332, 0.69194421]))
('Correlation Coef.: ', 0.8272891033747191)
Mean squared error: 0.01
Coefficient of determination: 0.68
Mean absolute error: 0.07
Deep regressor traning and predicting finished. Time spent = 0.75s.

---------MLP REG. RESULTS-----------

('Real Values : ', array([0.38, 0.3 , 0.23, 0.26, 0.18, 0.28, 0.31, 0.72, 0.6 , 0.38, 0.3 ,
       0.31, 0.3

In [ ]:
# Plot outputs
plt.scatter(X_test[:,0], y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor

X_train = np.reshape(X_train, (X_train.shape[0], 13))
X_test = np.reshape(X_test, (X_test.shape[0], 13))
print(type(y_train))
y_train = np.reshape(y_train, (234,1))

print(y_train.shape)

#kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(random_state=0).fit(X_train, y_train)
y_pred = gpr.predict(X_test)

y_pred = np.reshape(y_pred, (y_pred.shape[0],1))
#y_pred = net_train_and_predict(X_train, y_train, X_test,10, SEED_VAL)
#print(y_pred)
print("Real Values : ",y_test)
print("Predictions : ", y_pred) 
#regr = RandomForestRegressor(n_estimators = 100, random_state=42).fit(X_train, y_train)


r = np.corrcoef(y_test, y_pred)

# Correlation Coefficients
print('Correlation Coef.: ',r[0, 1] )

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))
print("gpr :  ", gpr.score(X_train, y_train))
print('Mean absolute error: %.2f'
      % mean_absolute_error(y_test, y_pred))
# Plot outputs
plt.scatter(X_test[:,0], y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()